# Experience

This experience shows an approach used to perform Time-Series data analysis involving Machine Learning.

Considerations:
- Multiple features to be analyzed;
- Possible correlation between features;
- Unlabeled data;
- (Un)supervised learning;

In [1]:
import pandas as pd

The code bellow was extracted from a module, opentsdb client, presented in Graphy.
This function retrieves data from OpenTSDB using the REST API defined by the database developers. To retrieve data, the metric name, start timestamp and end timestamp must be provided.

In [2]:
import requests
import json

opentsdb_address = 'http://127.0.0.1:4242/'
api_query = 'api/query'

def get_metrics(name: str, start_timestamp: int, end_timestamp: int) -> dict:
    """
    Gets the metrics from OpenTSDB.

    :param name: The name of the metrics.
    :param start_timestamp: The start unix timestamp of the metric.
    :param end_timestamp: The end unix timestamp of the metric.
    :return: The metrics as a dictionary if success, None otherwise.
    """
    json_body = {
        "start": start_timestamp,
        "end": end_timestamp,
        "queries": [{"aggregator": "sum", "metric": name},
                    {"aggregator": "sum", "tsuids": ["000001000002000042", "000001000002000043"]}]
    }

    data = None
    try:
        response = requests.post(opentsdb_address + api_query, data=json.dumps(json_body),
                                 headers={'content-type': 'application/json'})
        if response.status_code == 200:
            response_text = json.loads(response.text)
            if response_text:
                data = response_text[0].get('dps', None)
        return data
    except ConnectionError as ex:
        logger.error('{}: {}'.format(type(ex), ex))
        sys.exit(status=1)

# Data to be analyzed

The data is gathered from a Time-Series database (OpenTSDB). In this case we get some metrics from this kind of database to perform the analysis.

We are using the OUTER JOIN method to merge data from multiple features. This merge method preserves the data points and fills the missing values with NaN (Missing values).

In [3]:
start_timestamp = 1530057600
end_timestamp = 1530316800

service_name = 'nova-api-cascading'

metric_name = 'huawei.call_count.{}'.format(service_name)
metric_name_2 = 'huawei.status_code.{}.2XX'.format(service_name)
metric_name_3 = 'huawei.status_code.{}.4XX'.format(service_name)
metric_name_4 = 'huawei.status_code.{}.5XX'.format(service_name)

metrics = get_metrics(metric_name, start_timestamp, end_timestamp)
metrics_2 = get_metrics(metric_name_2, start_timestamp, end_timestamp)
metrics_3 = get_metrics(metric_name_3, start_timestamp, end_timestamp)
metrics_4 = get_metrics(metric_name_4, start_timestamp, end_timestamp)

# print(len(metrics.items()))
# print(len(metrics_2.items()))
# print(len(metrics_3.items()))
# print(len(metrics_4.items()))

df_1 = pd.DataFrame(metrics.items(), columns=['time', 'call_count'])
df_2 = pd.DataFrame(metrics_2.items(), columns=['time', 'status_code_2xx'])
df_3 = pd.DataFrame(metrics_3.items(), columns=['time', 'status_code_4xx'])
df_4 = pd.DataFrame(metrics_4.items(), columns=['time', 'status_code_5xx'])

df = pd.merge(df_1, df_2, how='outer')
df = pd.merge(df, df_3, how='outer')
df = pd.merge(df, df_4, how='outer')

print('\nData info:\n{}'.format(df.info()))

print('\nData:\n{}'.format(df))

print('\nMissing values counting:\n{}'.format(df.isna().sum()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1341 entries, 0 to 1340
Data columns (total 5 columns):
time               1341 non-null object
call_count         1341 non-null int64
status_code_2xx    1274 non-null float64
status_code_4xx    1168 non-null float64
status_code_5xx    8 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 62.9+ KB

Data info:
None

Data:
            time  call_count  status_code_2xx  status_code_4xx  \
0     1530137700         758         0.280543         0.719457   
1     1530138135         271         0.388571         0.611429   
2     1530138150         676         0.295000         0.705000   
3     1530138165         405         0.222222         0.777778   
4     1530138195          82         0.142857         0.857143   
5     1530138210          82         0.142857         0.857143   
6     1530138300         758         0.280543         0.719457   
7     1530138600         758         0.280543         0.719457   
8     15301406

# Plotting

In this code block, for visualization purposes, we plot some data retrieved from the time-series database.

As we can see, there are missing points in some charts. To perform a good analysis, this problem must be solved.

In [4]:
from matplotlib import pyplot as plt

df.plot(x='time', y='call_count', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('Call count')
plt.title('Time Series of Call Count')

df.plot(x='time', y='status_code_2xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_2xx')
plt.title('Time Series of status_code_2xx')

df.plot(x='time', y='status_code_4xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_4xx')
plt.title('Time Series of status_code_4xx')

df.plot(x='time', y='status_code_5xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_5xx')
plt.title('Time Series of status_code_5xx')

Text(0.5, 1.0, 'Time Series of status_code_5xx')

# Missing Values -- Pain :'(

Missing values are a pain in data analysis because we can not perform a good analysis of the data with missing values, and sometimes we do not know clearly what is the best option to fulfill the data. This values are represented by NaN in the dataframes. We can not use any clustering in dataframes with this kind of values.

Approaches:
- Remove rows with missing values (Degrades the overall data and may result in insufficient data);
- Impute missing values (May be dangerous because it introduces "wrong" observations);

For this example, and because we have a lot of missing values, the best option is to impute missing values, otherwise, if we remove the rows with the missing values it would cause insufficiency in the data which in turn results in inefficient training of the machine learning model.

Now, there are several ways we can perform the imputation:
- A constant value that has meaning within the domain, such as 0, distinct from all other values;
- A value from another randomly selected record;
- A mean, median or mode value for the column;
- Linear interpolation;
- A value estimated by another machine learning model.

[How to handle missing data in Time-Series](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)

To know better the method to use to fulfill the missing values, we perform a decomposition of data to perceive how is the Trend, Seasonality and Residuals in our data.

From the charts displayed bellow, we can notice that our data only represents trends. From the link provided above, through the path "Handling Missing Data --> Imputation --> Time-Series Problem --> Data with Trend and without Seasonality", we can select "Linear Interpolation" as the best method to fulfill our data.

In [5]:
df_copy = df.copy()

In [6]:
from random import randrange
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df['call_count'].dropna(), model='additive', freq=1)
result.plot()
pyplot.show()

result = seasonal_decompose(df['status_code_2xx'].dropna(), model='additive', freq=1)
result.plot()
pyplot.show()

result = seasonal_decompose(df['status_code_4xx'].dropna(), model='additive', freq=1)
result.plot()
pyplot.show()

result = seasonal_decompose(df['status_code_5xx'].dropna(), model='additive', freq=1)
result.plot()
pyplot.show()

ModuleNotFoundError: No module named 'statsmodels'

# Missing values fulfill

As explained before, with time-series data decomposed and showing only sines of Trend without Seasonality, the best option to fulfill data is to perform a linear interpolation on the data. From this, bellow is a piece of code that applies linear interpolation to the dataframe.

In [ ]:
df = df_copy.copy()

df = df.interpolate(method ='linear')

df_copy.plot(x='time', y='status_code_2xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_2xx')
plt.title('Time Series of status_code_2xx (MISSING_VALUES)')

df.plot(x='time', y='status_code_2xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_2xx')
plt.title('Time Series of status_code_2xx (COMPLETE)')

df.plot(x='time', y='status_code_4xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_4xx')
plt.title('Time Series of status_code_4xx (COMPLETE)')

df.plot(x='time', y='status_code_5xx', figsize=(12,6))
plt.xlabel('Timestamps')
plt.ylabel('status_code_5xx')
plt.title('Time Series of status_code_5xx (COMPLETE)')

In [ ]:
import numpy as np

df['status'] = np.where(df['status_code_2xx'] > df['status_code_4xx'] + df['status_code_5xx'], 1, 0)

df['status'].value_counts()

In [ ]:
from sklearn.cluster import KMeans

data = df[['call_count', 'status']]
n_cluster = range(1, 10)
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(n_cluster, scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show();

In [ ]:
from sklearn.cluster import KMeans

data = df[['call_count', 'status_code_2xx', 'status_code_4xx', 'status_code_5xx']]
n_cluster = range(1, 10)
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(n_cluster, scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show();

In [ ]:
df.info()

X = df[['call_count', 'status_code_2xx', 'status_code_4xx', 'status_code_5xx']]
X = X.reset_index(drop=True)
km = KMeans(n_clusters=3.5)
km.fit(X)
km.predict(X)
labels = km.labels_
#Plotting
fig = plt.figure(1, figsize=(7,7))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
ax.scatter(X.iloc[:,0], X.iloc[:,1], X.iloc[:,2],
          c=labels.astype(np.float), edgecolor="k")
ax.set_xlabel("price_usd")
ax.set_ylabel("srch_booking_window")
ax.set_zlabel("srch_saturday_night_bool")
plt.title("K Means", fontsize=14);

In [ ]:
from sklearn.preprocessing import StandardScaler

data = df[['call_count', 'status']]
X = data.values
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key = lambda x: x[0], reverse= True)
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance

plt.figure(figsize=(10, 5))
plt.bar(range(len(var_exp)), var_exp, alpha=0.3, align='center', label='individual explained variance', color = 'g')
plt.step(range(len(cum_var_exp)), cum_var_exp, where='mid',label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show();

In [ ]:
from sklearn.preprocessing import StandardScaler

data = df[['call_count', 'status_code_2xx', 'status_code_4xx', 'status_code_5xx']]
X = data.values
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key = lambda x: x[0], reverse= True)
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance

plt.figure(figsize=(10, 5))
plt.bar(range(len(var_exp)), var_exp, alpha=0.3, align='center', label='individual explained variance', color = 'g')
plt.step(range(len(cum_var_exp)), cum_var_exp, where='mid',label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show();

In [ ]:
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.set_value(i, np.linalg.norm(Xa-Xb))
    return distance

outliers_fraction = 0.01
# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
distance = getDistanceByPoint(data, kmeans[8])
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
# anomaly1 contain the anomaly result of the above method Cluster (0:normal, 1:anomaly) 
df['anomaly1'] = (distance >= threshold).astype(int)

# visualisation of anomaly with cluster view
fig, ax = plt.subplots(figsize=(10,6))
colors = {0:'blue', 1:'red'}
ax.scatter(df['call_count'], df['status'], c=df["anomaly1"].apply(lambda x: colors[x]))
plt.xlabel('principal feature1')
plt.ylabel('principal feature2')
plt.show();

In [ ]:
df = df.sort_values('time')
fig, ax = plt.subplots(figsize=(10,6))
a = df.loc[df['anomaly1'] == 1, ['time', 'call_count']] #anomaly

ax.plot(df['time'], df['call_count'], color='blue', label='Normal')
ax.scatter(a['time'],a['call_count'], color='red', label='Anomaly')
plt.xlabel('Timestamp')
plt.ylabel('Call count')
plt.legend()
plt.show();

fig, ax = plt.subplots(figsize=(10,6))
a = df.loc[df['anomaly1'] == 1, ['time', 'status']] #anomaly

ax.plot(df['time'], df['status'], color='blue', label='Normal')
ax.scatter(a['time'],a['status'], color='red', label='Anomaly')
plt.xlabel('Timestamp')
plt.ylabel('Status')
plt.legend()
plt.show();